In [ ]:
%run circuit_dynamics_init.py # all needed functions 
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format ='retina' #plot high-res img 
%load_ext line_profiler # line profiler

### Possible parallel strategy: 

i) because the order dot product of different $U_n$ matrices with psi
is not relevant, we can generate $L/2$ #s of $U_n$ parallelly and send back to the master rank.
but we still have to apply the dot product sequentially in order to update the wavefunction 
in a recursive manner
        
ii) using a more low level approach by paralleling the kronecker product using MPI.
        
iii) Since we know the sparse matrix will be block diagonal 
with identical blocks, i.e., for each i, un = np.repeat\[blk, $2^{2*i}$ \],
where all blocks take the form $\textrm{blk}=\textrm{kron}(u, \textrm{identity_mat}(2^{(L-2i-2)}$)


it's easy to perform parallel dot operations by extracting different portions of 
the wavefunction and then join the vector back, 
i.e., new_vec = join(dot(blk, vec\[index with the length of blk\]))
        
Note: the sequential version of method (iii) doesn't work well
        
iv) The kronecker product between two matrices A and B can be obtained by 
the outer product of A_row with B_row, say, Outer\[$A_1, B_1$\] gives the first row of 
the resulting matrix. So we can compute the outer product in parallel 
(also calculate the corresponding dot product with the wavefunction at the same time)
and gather the resulting dot product, thus eliminating the need to compute the whole 
large sparse matrix.      

v) Simplest one: after evolving around $2 L$ time steps, the wavefunction reaches a steady state.
We can use the this wavefunction as an input to multiple nodes using MPI to gather 
enough data points (embarrassing parallel)

#### Import physical parameters and initialize wavefunction 

In [ ]:
# reading parameters from file
para = open('para_haar.txt', 'r')
para = para.readlines()
# the paramters are system size, measurement probability and discrete time steps
L,  pro, time = int(para[0]), float(para[1]), int(para[2])

# system partition
# with PBC, we partition system into 4 parts where a and b separated by c1 and c2
# c1 and c2 are effectively connected, so the system is composed of A, B and C
lc1, la, lb = int(np.floor(L/8)), int(np.floor(L/4)), int(np.floor(L/4))
lc2 = L-lc1-la-lb
# pack the partition into array
part = np.array([L, la, lb, lc1, lc2], dtype="int64")

# initializing wavefunctions
p1 = np.ones(1)
p2 = np.zeros(2**L-1,dtype='c16')
# a product state with all spins align up
psi = np.concatenate((p1,p2),axis=0).T
# Pauli z-matrix
sz = sparse.coo_matrix(np.array([[1, 0],[0, -1]]))

In [ ]:
# time evolution consists of random unitaries + projective measurement
def evo(steps, wave, prob, L, n, partition):
    von=np.zeros(steps, dtype='float64') # von-Neumann entropy
    renyi=np.zeros(steps, dtype='float64') # Renyi entropy
    neg = np.zeros(steps, dtype='float64') # logarithmic negativity
    mut = np.zeros(steps, dtype='float64') # mutual information using von-Neumann entropy
    mutr = np.zeros(steps, dtype='float64') # mutual information in terms of Renyi entropy
    
    for t in range(steps):
        # evolve over odd links
        for i in range(L//2):
            wave = unitary(wave, i, L)     
        
        # measurement layer
        for i in range(L):
            wave = measure(wave, prob, i, L)

        # before evolve on even link, we need to rearrange indices first to accommodate the boundary condition PBC
        wave = np.reshape(wave,(2, 2**(L-2),2))
        # move the last site into the first one such that the unitaries can connect the 1st and the last site
        wave = np.moveaxis(wave,-1,0)
        wave = wave.flatten()
        
        # evolve over even links
        for i in range(L//2):
            wave = unitary(wave, i, L)  

        #shift the index back to the original order after evolution
        wave = np.reshape(wave,(2, 2, 2**(L-2)))
        wave = np.moveaxis(wave,-1,0)
        wave = np.moveaxis(wave,-1,0).flatten()

        #measurement layer
        for i in range(L):
            wave = measure(wave, prob, i, L)
       
        result = ent(wave, n, L, L//2)
        von = result[0]
        ren = result[1]
        result = logneg(wave, n, partition)
        neg[t] = result[0]
        mut[t] = result[1]
        mutr[t] = result[2]

    return von, ren, neg, neg2, mut, mutr

In [ ]:
res = evo(time, psi, pro, L, 2, part)